In [1]:
import requests
from IPython.display import JSON

In [1]:
#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2021-11-25T08:00/2021-11-25T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': "e5fd3f0d20msh33512308774393cp1f8519jsn3f904e2f39d1"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [11]:
JSON(response.text)

<IPython.core.display.JSON object>

In [24]:
flight_arrivals = response.json()

for flight in flight_arrivals:
flight_arrivals['arrivals'][0]['arrival']['scheduledTimeLocal']
flight_arrivals['arrivals'][0]['number']
flight_arrivals['arrivals'][0]['arrival']['terminal']
flight_arrivals['arrivals'][0]['departure']['airport']['name']
flight_arrivals['arrivals'][0]['aircraft']['model']
flight_arrivals['arrivals'][0]['airline']['name']


'Austrian'

In [47]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTimeLocal'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,Scheduled_arrival_time,flight_number,from,airline,aircraft
0,2021-11-25 08:55+01:00,OS 223,Vienna,Austrian,Embraer 195
0,2021-11-25 09:00+01:00,JU 350,Belgrade,Air Serbia,Airbus A320
0,2021-11-25 08:55+01:00,LH 174,Frankfurt-am-Main,Lufthansa,Airbus A320
0,2021-11-25 08:55+01:00,AF 7354,Paris,Air France,Airbus A320
0,2021-11-25 09:30+01:00,FR 197,Budapest,Ryanair,Boeing 737-800


## Flight data functions for the Cloud 

In [2]:
import pandas as pd
from keys import *
import datetime
import requests
from IPython.display import JSON

In [3]:
#be careful not to run too often as there is a maximum of 200 api calls per month

def get_flight_data(icao):
    today = datetime.date.today()
    tomorrow = str(today + datetime.timedelta(days=1))
    
    hours = [["00:00","11:59"],["12:00","23:59"]]
    flight_data = []
    for time in hours:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

        headers = {
            'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
            'x-rapidapi-key': flights_key
            }

        flight_data.append(requests.request("GET", url, headers=headers, params=querystring).json())
    return flight_data

In [7]:
test = get_flight_data('EDDB')

In [19]:
test[1]['arrivals'][0]['departure']['airport']['name']

'Helsinki'

In [23]:
JSON(test)

<IPython.core.display.JSON object>

In [4]:
flight_data = get_flight_data('EDDB')

In [37]:
JSON(flight_data)

<IPython.core.display.JSON object>

In [11]:
#flight_arrivals = flight_data.json()

icao = 'EDDB'

def flight_detail_extraction(flights):
    
    try: terminal = flights['arrival']['terminal']
    except: terminal = None
    
    try: aircraft = flights['aircraft']['model']
    except: aircraft = None    
    
    flights_df = pd.json_normalize({
        'scheduled_arrival_time': flights['arrival']['scheduledTimeLocal'],
        'flight_number': flights['number'],
        'dep_airport' : flights['departure']['airport']['name'],
        'terminal' : terminal,
        'airline' : flights['airline']['name'],
        'aircraft' : aircraft,
        'icao_code' : icao
    })
    return flights_df 

def create_flight_df(flight_data):
    flight_arrivals = []

    for j in flight_data: 
        new_flights = pd.concat([flight_detail_extraction(flight) for flight in j['arrivals']])
        flight_arrivals.append(new_flights)

    flight_arrivals = pd.concat(flight_arrivals) 
    return flight_arrivals

In [12]:
flight_arrivals = create_flight_df(flight_data)
flight_arrivals.head()

,scheduled_arrival_time,flight_number,dep_airport,terminal,airline,aircraft,icao_code
0,2021-12-03 07:25+01:00,EW 12,Cologne,1,Eurowings,Boeing 737-800,EDDB
0,2021-12-03 07:35+01:00,EW 9048,Duesseldorf,1,Eurowings,Airbus A319,EDDB
0,2021-12-03 07:55+01:00,LH 170,Frankfurt-am-Main,1,Lufthansa,Airbus A321,EDDB
0,2021-12-03 07:40+01:00,U2 5632,Bâle/Mulhouse,1,easyJet,Airbus A320,EDDB
0,2021-12-03 07:30+01:00,W6 4515,Varna,0,Wizz Air,Airbus A320,EDDB
